In [ ]:
import os
import pandas as pd

# Define folder paths for ACS data
folder_paths = ["data/DP03", "data/DP02"]

# Load cleaned Zillow data
zillow_df = pd.read_csv("data/zillow_with_county.csv")

# Dictionary to store merged data for each year
all_years_data = []

# Iterate over each folder (DP03 for income, DP02 for education)
for folder_path in folder_paths:
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):  # Ensure it's a CSV file
            file_path = os.path.join(folder_path, file)
            year = int(file.split(".")[0][-4:])  # Extract year from filename
            
            # Load the file
            acs_df = pd.read_csv(file_path)

            # Standardize ZCTA column name
            acs_df.rename(columns={"geography": "ZCTA"}, inplace=True)
            acs_df["ZCTA"] = acs_df["ZCTA"].astype(str)

            # Extract relevant columns
            if folder_path == "DP03":  # Income data
                acs_df.rename(columns={"DP03_0062E": "median_household_income"}, inplace=True)
                acs_income = acs_df[["ZCTA", "median_household_income"]]
            elif folder_path == "DP02":  # Education data
                acs_df.rename(columns={"DP02_0064PE": "college_educated_population"}, inplace=True)
                acs_education = acs_df[["ZCTA", "college_educated_population"]]

    # Merge with Zillow data for the corresponding year
    merged_df = zillow_df[zillow_df["year"] == year].merge(acs_income, on="ZCTA", how="left")
    merged_df = merged_df.merge(acs_education, on="ZCTA", how="left")

    # Append the merged dataset for this year
    all_years_data.append(merged_df)

# Concatenate all years into a single DataFrame
final_df = pd.concat(all_years_data)

# Ensure data is sorted by ZCTA and Year before computing rolling averages
final_df = final_df.sort_values(by=["ZCTA", "year"])

# Compute 5-year rolling averages
final_df["home_price_5yr_avg"] = final_df.groupby("ZCTA")["median_home_value"].transform(lambda x: x.rolling(5, min_periods=1).mean())
final_df["income_5yr_avg"] = final_df.groupby("ZCTA")["median_household_income"].transform(lambda x: x.rolling(5, min_periods=1).mean())
final_df["college_edu_5yr_avg"] = final_df.groupby("ZCTA")["college_educated_population"].transform(lambda x: x.rolling(5, min_periods=1).mean())

# Calculate county-level rolling averages for comparison
final_df["county_home_price_5yr_avg"] = final_df.groupby(["NAMELSAD_COUNTY_20", "year"])["median_home_value"].transform(lambda x: x.rolling(5, min_periods=1).mean())
final_df["county_income_5yr_avg"] = final_df.groupby(["NAMELSAD_COUNTY_20", "year"])["median_household_income"].transform(lambda x: x.rolling(5, min_periods=1).mean())

# Compute percentage change relative to county averages
final_df["home_price_growth"] = (final_df["home_price_5yr_avg"] - final_df["county_home_price_5yr_avg"]) / final_df["county_home_price_5yr_avg"]
final_df["income_growth"] = (final_df["income_5yr_avg"] - final_df["county_income_5yr_avg"]) / final_df["county_income_5yr_avg"]
final_df["college_edu_growth"] = final_df.groupby("ZCTA")["college_edu_5yr_avg"].pct_change()

# Identify top 20% threshold for home price & income growth
home_price_threshold = final_df["home_price_growth"].quantile(0.8)
income_threshold = final_df["income_growth"].quantile(0.8)

# Gentrification criteria (meeting at least 2 out of 3 conditions)
final_df["gentrified"] = ((final_df["home_price_growth"] > home_price_threshold).astype(int) +
                           (final_df["income_growth"] > income_threshold).astype(int) +
                           (final_df["college_edu_growth"] >= 0.10).astype(int)) >= 2

# Create a binary column for each year (2016-2023) indicating gentrification
for year in range(2016, 2024):
    final_df[f"gentrified_{year}"] = (final_df["year"] == year) & (final_df["gentrified"])

# Save the processed dataset
final_df.to_csv("processed_gentrification_data.csv", index=False)

print("Gentrification analysis completed. File saved as 'processed_gentrification_data.csv'.")

